In [1]:
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
from sklearn import metrics
import tensorflow as tf
import os
import pickle
from tqdm.notebook import tqdm
from collections import defaultdict, OrderedDict
import requests

In [2]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [3]:
PATH = "/Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/"

In [4]:
listdir = os.listdir(os.path.join(PATH, "parsed_punc_content"))

In [5]:
titles = {}
bad_files = []
for file in tqdm(listdir, total=len(listdir)):
    try:
        docid = file.split("_")[-1].split(".")[0]
        with open(os.path.join(PATH, "parsed_punc_content", file), "rb") as f:
            titles[docid] = pickle.load(f)[1]
    except:
        bad_files.append(file)

In [6]:
with open(os.path.join(PATH, "parsed_punc_content", "parsed_punc_1.pkl"), "rb") as f:
    print(pickle.load(f))

('', 'Хорватия', '', ['', '', '', 'Хорватия 32383 просмотров ', '', ''])


In [7]:
bad_files

[]

In [8]:
len(titles)

38114

In [16]:
bad_parsed = {}
for file in tqdm(bad_files, total=len(bad_files)):
    docid = file.split("_")[1].split(".")[0]
    with open(os.path.join(PATH, "parsed_content", file), "rb") as f:
        bad_parsed[docid] = pickle.load(f)[1]

In [17]:
bad_parsed

{}

In [18]:
for key, value in bad_parsed.items():
    titles[key] = value

In [37]:
len(titles)

38114

In [9]:
with open(PATH + "queries.numerate.txt", "r") as f:
    queries = f.read().split("\n")

In [10]:
with open(PATH + "sample.technosphere.ir1.textrelevance.submission.txt", "r") as f:
    sample = f.read().split("\n")

In [11]:
query_docid = defaultdict(list)
for item in sample[1:]:
    if item:
        qid = item.split(",")[0]
        docid = item.split(",")[1]
        query_docid[qid].append(docid)

In [15]:
queries_to_map = []

for query in queries[:-1]:
    queries_to_map.append(query.split("\t")[1])

In [16]:
def spellcheck(text):
    payload = {"text": text}
    response = requests.get("https://speller.yandex.net/services/spellservice.json/checkText",
                            params=payload).json()
    
    fixed_text = text
    for item in response:
        fixed_text = fixed_text.replace(item["word"], item["s"][0])
    return fixed_text

In [17]:
fix_queries_to_map = [spellcheck(query) for query in tqdm(queries_to_map, total=len(queries_to_map))]

KeyboardInterrupt: 

In [32]:
queries_embeddings = embed(fix_queries_to_map)

In [33]:
import os

In [34]:
with open(os.path.join(PATH, "queries_embeddings"), "wb") as f:
    pickle.dump(queries_embeddings, f)

In [12]:
with open(os.path.join(PATH, "queries_embeddings"), "rb") as f:
    queries_embeddings = pickle.load(f)

In [49]:
titles_list = []
for i in range(1, len(titles) + 1):
    titles_list.append(titles[str(i)])

In [50]:
len(titles_list)

38114

In [20]:
!mkdir /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/title_punc_embeddings

In [27]:
with open(os.path.join(PATH, "titles_punc.pkl"), "wb") as f:
    pickle.dump(titles, f)

In [8]:
with open(os.path.join(PATH, "titles_punc.pkl"), "rb") as f:
    titles = pickle.load(f)

In [13]:
titles

{'18899': 'нарисовать череп в CorelDraw',
 '11103': 'по следам самого Создателя Проза Приключения ',
 '680': 'Renault Symbol, ваше мнение — форум Wroom;ru',
 '18641': 'Ольга Бугрова: «Работаем индивидуально с каждым заявителем» sotsproekt-ryazan;ru',
 '26977': 'Единственный крест Храм в честь Священномученика Андроника, архиепископа Пермского',
 '17572': 'На прием к гинекологу видео порно смотреть онлайн',
 '28782': 'Как убрать шишки косточки на ногах форум Woman;ru',
 '858': 'обильное теплое питье, хорошо ромашковый час; фенистил т;к; сухой кашель может обернуться ларингитом , фарингосепт или что-то похожее сосательно-антисептическое',
 '6812': 'Хронологическая таблица жизни лескова',
 '13714': 'Где самый недорогой отдых за границей - allWomens',
 '32902': 'Скачать драйвер для звуковой карты бесплатно, установка драйвера для аудио карты Windows 7, 8, XP',
 '15365': 'The Sims 4: Использование кода MoveObjects - RoleSim;Net',
 '15403': 'Что делать, если пересолил',
 '24806': '1С:Предпри

In [14]:
bad_queries = []
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    try:
        titles_tmp = [titles[docid][:500] for docid in docids]
        titles_emb_tmp = embed(titles_tmp)
        with open(os.path.join(PATH, "title_punc_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(titles_emb_tmp, f)
    except:
        print(query)
        bad_queries.append(query)

In [52]:
listdir = os.listdir(os.path.join(PATH, "title_punc_embeddings"))

In [53]:
len(listdir)

399

In [18]:
with open(os.path.join(PATH, "queries_embeddings"), "rb") as f:
    queries_embeddings = pickle.load(f)

In [61]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_punc_embeddings/{query}.pkl"), "rb") as f:
        data = pickle.load(f)
    ranged_query_docid[query] = np.array(docids)[np.argsort(metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data)[0])]

In [62]:
ranged_query_docid

{'1': array(['28', '78', '53', '74', '66', '7', '58', '85', '75', '51', '57',
        '44', '60', '17', '68', '3', '94', '54', '88', '67', '83', '96',
        '56', '70', '90', '24', '77', '73', '82', '46', '47', '91', '62',
        '61', '22', '95', '18', '72', '64', '26', '1', '11', '76', '39',
        '45', '9', '14', '34', '84', '12', '37', '42', '13', '93', '80',
        '41', '65', '48', '35', '36', '43', '71', '31', '2', '16', '33',
        '59', '15', '55', '79', '27', '52', '29', '8', '32', '87', '30',
        '4', '63', '89', '21', '38', '25', '19', '23', '49', '20', '92',
        '69', '6', '10', '50', '40', '5', '81', '86'], dtype='<U2'),
 '2': array(['154', '125', '153', '192', '149', '103', '131', '165', '156',
        '99', '150', '166', '186', '179', '137', '101', '185', '114',
        '183', '177', '102', '167', '141', '164', '180', '171', '181',
        '145', '116', '152', '133', '118', '146', '159', '117', '143',
        '100', '155', '182', '147', '157', '172', '12

In [65]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [66]:
ranged_query_docid

{'1': ['28',
  '78',
  '53',
  '74',
  '66',
  '7',
  '58',
  '85',
  '75',
  '51',
  '57',
  '44',
  '60',
  '17',
  '68',
  '3',
  '94',
  '54',
  '88',
  '67',
  '83',
  '96',
  '56',
  '70',
  '90',
  '24',
  '77',
  '73',
  '82',
  '46',
  '47',
  '91',
  '62',
  '61',
  '22',
  '95',
  '18',
  '72',
  '64',
  '26',
  '1',
  '11',
  '76',
  '39',
  '45',
  '9',
  '14',
  '34',
  '84',
  '12',
  '37',
  '42',
  '13',
  '93',
  '80',
  '41',
  '65',
  '48',
  '35',
  '36',
  '43',
  '71',
  '31',
  '2',
  '16',
  '33',
  '59',
  '15',
  '55',
  '79',
  '27',
  '52',
  '29',
  '8',
  '32',
  '87',
  '30',
  '4',
  '63',
  '89',
  '21',
  '38',
  '25',
  '19',
  '23',
  '49',
  '20',
  '92',
  '69',
  '6',
  '10',
  '50',
  '40',
  '5',
  '81',
  '86'],
 '2': ['154',
  '125',
  '153',
  '192',
  '149',
  '103',
  '131',
  '165',
  '156',
  '99',
  '150',
  '166',
  '186',
  '179',
  '137',
  '101',
  '185',
  '114',
  '183',
  '177',
  '102',
  '167',
  '141',
  '164',
  '180',
  '171

In [67]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_10_3.txt", "w") as f:
    f.write("\n".join(submit))

In [68]:
len(submit)

38773

In [67]:
counter = 0
for query in ranged_query_docid:
    counter += len(ranged_query_docid[query])
print(counter)

38772


# Add meta tags

In [56]:
listdir = os.listdir(os.path.join(PATH, "parsed_punc_content"))

In [23]:
meta = {}
bad_files = []
for file in tqdm(listdir, total=len(listdir)):
    try:
        docid = file.split("_")[-1].split(".")[0]
        with open(os.path.join(PATH, "parsed_punc_content", file), "rb") as f:
            meta[docid] = pickle.load(f)[2]
    except:
        bad_files.append(file)

In [24]:
len(meta)

38114

In [57]:
!mkdir /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/meta_punc_embeddings

mkdir: /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/meta_punc_embeddings: File exists


In [26]:
with open(os.path.join(PATH, "meta_punc.pkl"), "wb") as f:
    pickle.dump(meta, f)

In [58]:
bad_queries = []
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    try:
        meta_tmp = [meta[docid][:1000] for docid in docids]
        meta_emb_tmp = embed(meta_tmp)
        with open(os.path.join(PATH, "meta_punc_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(meta_emb_tmp, f)
    except:
        print(query)
        bad_queries.append(query)

In [59]:
bad_queries

[]

In [60]:
len(os.listdir(os.path.join(PATH, "meta_punc_embeddings")))

399

In [88]:
bad_queries = ["6", "8"]
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    if query in bad_queries:
#     try:
        meta_tmp = [meta[docid][:500] for docid in docids]
        meta_emb_tmp = embed(meta_tmp)
        with open(os.path.join(PATH, "meta_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(meta_emb_tmp, f)
#     except:
#         print(query)
#         bad_queries.append(query)

In [114]:
len(os.listdir(os.path.join(PATH, "meta_embeddings")))

399

In [69]:
with open(os.path.join(PATH, "meta_punc_embeddings", f"{6}.pkl"), "rb") as f:
    data = pickle.load(f)

In [70]:
data.shape

TensorShape([108, 512])

In [37]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f:
        data_title = pickle.load(f)
    
    with open(os.path.join(PATH, f"meta_embeddings/{query}.pkl"), "rb") as f:
        data_meta = pickle.load(f)
        
    title_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_title)[0]
    meta_score = 0 #metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_meta)[0]
    
    ranged_query_docid[query] = np.array(docids)[np.argsort(0.7 * title_score + 0.3 * meta_score)]

In [38]:
ranged_query_docid

{'1': array(['28', '78', '53', '74', '66', '58', '7', '75', '67', '57', '51',
        '85', '60', '44', '70', '17', '94', '54', '3', '56', '83', '88',
        '96', '68', '72', '90', '82', '24', '47', '77', '73', '61', '62',
        '91', '18', '64', '95', '11', '46', '22', '1', '39', '26', '14',
        '76', '45', '34', '84', '9', '12', '37', '13', '35', '80', '65',
        '48', '93', '2', '41', '33', '31', '42', '36', '43', '71', '16',
        '59', '55', '52', '87', '27', '79', '15', '8', '29', '32', '4',
        '63', '25', '21', '38', '30', '89', '23', '19', '20', '49', '10',
        '69', '81', '50', '92', '6', '40', '5', '86'], dtype='<U2'),
 '2': array(['154', '125', '153', '192', '165', '149', '131', '103', '99',
        '156', '150', '166', '186', '185', '179', '137', '101', '114',
        '183', '141', '177', '102', '167', '164', '180', '171', '116',
        '145', '181', '133', '152', '159', '118', '182', '155', '143',
        '100', '135', '129', '190', '147', '157', '14

In [39]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [40]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_10_2.txt", "w") as f:
    f.write("\n".join(submit))

# Headers

In [48]:
listdir = os.listdir(os.path.join(PATH, "parsed_punc_content"))

headers = defaultdict(str)
bad_files = []
for file in tqdm(listdir, total=len(listdir)):
    try:
        docid = file.split("_")[-1].split(".")[0]
        with open(os.path.join(PATH, "parsed_punc_content", file), "rb") as f:
            for header in pickle.load(f)[3]:
                if header != "":
                    headers[docid] = header
                    break
                    
    except:
        bad_files.append(file)

In [49]:
with open(os.path.join(PATH, "headers_punc.pkl"), "wb") as f:
    pickle.dump(headers, f)

In [51]:
!mkdir /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/headers_punc_embeddings

In [71]:
bad_queries = []
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    try:
        headers_tmp = [headers[docid][:1000] for docid in docids]
        headers_emb_tmp = embed(headers_tmp)
        with open(os.path.join(PATH, "headers_punc_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(headers_emb_tmp, f)
        
    except:
        print(query)
        bad_queries.append(query)

In [72]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f: # titles with punc
        data_title = pickle.load(f)
    
    with open(os.path.join(PATH, f"headers_punc_embeddings/{query}.pkl"), "rb") as f: # headers without punc
        data_headers = pickle.load(f)
        
    title_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_title)[0]
    headers_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_headers)[0]
    
    ranged_query_docid[query] = np.array(docids)[np.argsort(0.7 * title_score + 0.3 * headers_score)]

In [73]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [74]:
ranged_query_docid

{'1': ['28',
  '74',
  '53',
  '78',
  '58',
  '66',
  '57',
  '44',
  '51',
  '7',
  '85',
  '75',
  '60',
  '70',
  '17',
  '83',
  '3',
  '54',
  '88',
  '56',
  '82',
  '96',
  '68',
  '73',
  '61',
  '94',
  '77',
  '90',
  '95',
  '18',
  '91',
  '47',
  '67',
  '11',
  '64',
  '22',
  '1',
  '72',
  '24',
  '62',
  '39',
  '26',
  '14',
  '84',
  '34',
  '46',
  '42',
  '13',
  '37',
  '35',
  '93',
  '76',
  '65',
  '12',
  '33',
  '2',
  '55',
  '43',
  '41',
  '45',
  '48',
  '9',
  '71',
  '16',
  '79',
  '59',
  '80',
  '52',
  '36',
  '8',
  '27',
  '31',
  '87',
  '32',
  '89',
  '15',
  '63',
  '4',
  '25',
  '29',
  '38',
  '20',
  '23',
  '21',
  '30',
  '49',
  '69',
  '19',
  '50',
  '81',
  '10',
  '6',
  '5',
  '92',
  '86',
  '40'],
 '2': ['125',
  '154',
  '153',
  '149',
  '165',
  '192',
  '103',
  '131',
  '156',
  '185',
  '99',
  '150',
  '166',
  '186',
  '179',
  '137',
  '101',
  '114',
  '183',
  '177',
  '102',
  '141',
  '180',
  '118',
  '167',
  '181

In [75]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_10_4.txt", "w") as f:
    f.write("\n".join(submit))

Заголовки с первого парсинга, headers со второго.

In [84]:
with open(os.path.join(PATH, "queries_all_large.pkl"), "rb") as f:
    queries_all = pickle.load(f)

In [76]:
with open(PATH + "queries.numerate_revised.txt", "r") as f:
    queries = f.read().split("\n")

In [77]:
queries_to_map = []

for query in queries[:-1]:
    queries_to_map.append(query.split("\t")[1])

In [79]:
queries_embeddings = embed(queries_to_map)

In [80]:
with open(os.path.join(PATH, "queries_embeddings_ext.pkl"), "wb") as f:
    pickle.dump(queries_embeddings, f)

In [81]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f: # titles with punc
        data_title = pickle.load(f)
    
    with open(os.path.join(PATH, f"headers_punc_embeddings/{query}.pkl"), "rb") as f: # headers without punc
        data_headers = pickle.load(f)
        
    title_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_title)[0]
    headers_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_headers)[0]
    
    ranged_query_docid[query] = np.array(docids)[np.argsort(0.7 * title_score + 0.3 * headers_score)]

In [82]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [83]:
ranged_query_docid

{'1': ['28',
  '74',
  '53',
  '78',
  '58',
  '66',
  '51',
  '44',
  '57',
  '7',
  '85',
  '70',
  '75',
  '60',
  '83',
  '88',
  '54',
  '3',
  '17',
  '68',
  '96',
  '82',
  '73',
  '94',
  '56',
  '61',
  '77',
  '91',
  '90',
  '95',
  '18',
  '11',
  '67',
  '47',
  '72',
  '22',
  '24',
  '1',
  '64',
  '62',
  '39',
  '26',
  '37',
  '84',
  '42',
  '46',
  '34',
  '14',
  '35',
  '76',
  '13',
  '12',
  '65',
  '93',
  '43',
  '33',
  '71',
  '16',
  '79',
  '2',
  '45',
  '80',
  '48',
  '41',
  '55',
  '36',
  '9',
  '8',
  '52',
  '59',
  '31',
  '87',
  '27',
  '25',
  '89',
  '32',
  '15',
  '29',
  '38',
  '4',
  '63',
  '23',
  '20',
  '21',
  '30',
  '50',
  '49',
  '10',
  '69',
  '81',
  '6',
  '19',
  '86',
  '5',
  '92',
  '40'],
 '2': ['125',
  '153',
  '149',
  '165',
  '154',
  '131',
  '150',
  '103',
  '192',
  '156',
  '185',
  '166',
  '99',
  '179',
  '101',
  '186',
  '180',
  '114',
  '141',
  '177',
  '137',
  '183',
  '167',
  '182',
  '164',
  '181

In [86]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_10_5.txt", "w") as f:
    f.write("\n".join(submit))